## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
#classifier.add(Convolution2D('自己設計參數'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
#classifier.add(Convolution2D('自己設計參數'))
classifier.add(Convolution2D(64,(3,3)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
#classifier.add(Dense('自己設計FC層參數')) #output_dim=100,activation=relu
#classifier.add(Dense(output_dim=100,activation='relu'))
classifier.add(Dense(units=100, activation='relu')) 

#輸出
#classifier.add(Dense(output_dim=10,activation='輸出函數應該用什麼？'))
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=20)

C:\Develop\Python\MarathonCVDL\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
Epoch 1/20
50000/50000 [==============================] - 17s 344us/step - loss: 1.4633 - accuracy: 0.5323
Epoch 2/20
50000/50000 [==============================] - 15s 296us/step - loss: 0.8938 - accuracy: 0.6921
Epoch 3/20
50000/50000 [==============================] - 14s 288us/step - loss: 0.7036 - accuracy: 0.7538
Epoch 4/20
50000/50000 [==============================] - 15s 293us/step - loss: 0.5584 - accuracy: 0.8051
Epoch 5/20
50000/50000 [==============================] - 15s 295us/step - loss: 0.4382 - accuracy: 0.8461
Epoch 6/20
50000/50000 [==============================] - 15s 296us/step - loss: 0.3477 - accuracy: 0.8775
Epoch 7/20
50000/50000 [==============================] - 15s 298us/step - loss: 0.2800 - accuracy: 0.9035
Epoch 8/20
50000/50000 [==============================] - 15s 307us/step - los

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.3585887e-04, 5.3443085e-07, 1.8245993e-05, 4.3906727e-01,
        1.1253517e-05, 1.4233269e-03, 3.3209464e-05, 5.5874884e-01,
        3.6106628e-04, 3.4579239e-07]], dtype=float32)